In [2]:
from elasticsearch import Elasticsearch

ES = Elasticsearch("http://localhost:9200")
ES.info().body

{'name': 'r1ck-MS-7D43',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'yOetF59CRYOwkd59On-lEw',
 'version': {'number': '8.6.2',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '2d58d0f136141f03239816a4e360a8d17b6d8f29',
  'build_date': '2023-02-13T09:35:20.314882762Z',
  'build_snapshot': False,
  'lucene_version': '9.4.2',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [19]:
!curl http://localhost:9200/_aliases?pretty=true

{
  "wiki-index" : {
    "aliases" : { }
  }
}


In [20]:
#!curl -X DELETE "localhost:9200/mrc-index?pretty"
!curl -X DELETE "localhost:9200/wiki-index?pretty"

{
  "acknowledged" : true
}


In [5]:
import json
import requests

In [6]:
url = "http://localhost:9200/wiki-index"

payload = json.dumps({
        'settings': {
            'similarity': {
                'bm25_similarity': {
                    'type': 'BM25',
                    'k1': 0.5,
                    'b': 0.5
                }
            }
        },
        'mappings': {
            '_source': {
                'includes': [
                    'context',
                    'segment_ctx'
                ],
                'excludes': [
                    'bm25_text'
                    'embedding'
                ]
            },
            'properties': {
                'context': {
                    'type': 'object',
                    'enabled': 'false'
                },
                'segment_ctx': {
                    'type': 'object',
                    'enabled': 'false'
                },
                'bm25_text': {
                    'type': 'text',
                    'similarity': 'bm25_similarity'
                },
                'embedding': {
                    'type': 'dense_vector',
                    'dims': 768,
                    'index': 'true',
                    'similarity': "dot_product",
                    'index_options': {
                        'type': 'hnsw',
                        'm': 32,
                        'ef_construction': 128
                    }

                }
            }
        }
    })
headers = {'Content-Type': 'application/json'}

response = requests.request("PUT", url, headers=headers, data=payload)

In [7]:
response.text

'{"acknowledged":true,"shards_acknowledged":true,"index":"wiki-index"}'

In [8]:
with open("corpus_wiki.json") as f:
    add_docs = json.load(f)

In [14]:
add_docs[0]

{'title': 'Vinylon',
 'text': 'Vinylon City Trong giai đoạn đầu của lịch sử Bắc Triều Tiên chính phủ dưới thời Kim Il Sung và ý thức hệ juche tự lực chính thức đã thúc đẩy ý tưởng rằng cách duy nhất để đạt được mục tiêu độc lập kinh tế là thông qua ngành công nghiệp nặng Do đó việc sản xuất vinylon được thực hiện như một bước tiến để phát triển Bắc Triều Tiên là một quốc gia công nghiệp hiện đại Với sự hấp dẫn như vậy đối với chủ nghĩa dân tộc chính phủ Bắc Triều Tiên đã huy động các công dân của mình để xây dựng và hỗ trợ một nhà máy vinylon mới được gọi là Vinylon City'}

In [9]:
from tqdm import tqdm
from utils import word_segment, bm25_tokenizer
from sentence_transformers import SentenceTransformer

/home/r1ck/VNPT/HybRES/src/vncorenlp
2023-03-21 09:49:53 INFO  WordSegmenter:24 - Loading Word Segmentation model


In [10]:
%cd ..

/home/r1ck/VNPT/HybRES/src


In [11]:
MODEL_PATH = "checkpoint-2022-06-28_04-25-02"
ENCODER = SentenceTransformer(MODEL_PATH, device="cuda")

In [12]:
from elasticsearch import helpers

In [17]:
chunk_size = 2000
add_docs = add_docs[:10000]
with tqdm(total=len(add_docs)) as pbar:
    idx = 0
    for start_idx in range(0, len(add_docs), chunk_size):
        end_idx = start_idx + chunk_size
        sub_docs = add_docs[start_idx:end_idx]
        segment_docs = []
        bm25_docs = []
        # process input text to bm25 and model format
        for doc in sub_docs:
            title = word_segment(doc["title"])
            content = word_segment(doc["text"])
            segment_docs.append({"passage_title": title, "passage_content": content})
            bm25_docs.append(bm25_tokenizer(title + " " + content))
        # encode doc to embedding
        print("#> Encode corpus...!")
        encode_docs = [doc["passage_title"] + " . " + doc["passage_content"] for doc in segment_docs]
        embeddings = ENCODER.encode(encode_docs, convert_to_numpy=True,
                                    normalize_embeddings=True, show_progress_bar=True)
        # add all features to index
        print("#> Add docs to index...!")
        bulk_data = []
        for doc, doc_segment, bm25_doc, embedding in zip(sub_docs, segment_docs, bm25_docs, embeddings):
            bulk_data.append({
                '_id': idx,
                'context': doc,
                'segment_ctx': doc_segment,
                'bm25_text': bm25_doc,
                'embedding': embedding
            })
            idx += 1
        helpers.bulk(ES, bulk_data, index="wiki-index")
        pbar.update(chunk_size)

  0%|                                                 | 0/10000 [00:00<?, ?it/s]

#> Encode corpus...!


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

#> Add docs to index...!


 20%|███████▍                             | 2000/10000 [00:18<01:12, 110.42it/s]

...

 80%|█████████████████████████████▌       | 8000/10000 [01:13<00:18, 108.88it/s]

#> Encode corpus...!


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

#> Add docs to index...!


100%|████████████████████████████████████| 10000/10000 [01:32<00:00, 107.76it/s]


In [18]:
ES.indices.refresh(index="wiki-index")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

In [25]:
add_docs[0]

{'title': 'Vinylon',
 'text': 'Vinylon City Trong giai đoạn đầu của lịch sử Bắc Triều Tiên chính phủ dưới thời Kim Il Sung và ý thức hệ juche tự lực chính thức đã thúc đẩy ý tưởng rằng cách duy nhất để đạt được mục tiêu độc lập kinh tế là thông qua ngành công nghiệp nặng Do đó việc sản xuất vinylon được thực hiện như một bước tiến để phát triển Bắc Triều Tiên là một quốc gia công nghiệp hiện đại Với sự hấp dẫn như vậy đối với chủ nghĩa dân tộc chính phủ Bắc Triều Tiên đã huy động các công dân của mình để xây dựng và hỗ trợ một nhà máy vinylon mới được gọi là Vinylon City'}

In [18]:
query = "ISOC là tổ chức gì"

In [20]:
question = word_segment(query)
top_k = 50
index = "wiki-index"
question_embedding = ENCODER.encode(query, show_progress_bar=False, normalize_embeddings=True)
lex_search = ES.search(index=index, size=top_k, query={"match": {"bm25_text": bm25_tokenizer(question)}}, source=False)
sparse_result = {hit["_id"]: hit["_score"] for hit in lex_search["hits"]["hits"]}
sem_search = ES.search(index=index, size=top_k, knn={"field": "embedding", "query_vector": question_embedding, "k": top_k, "num_candidates": 256}, source=False)
dense_result = {hit["_id"]: hit["_score"] for hit in sem_search["hits"]["hits"]}

In [21]:
sem_search

ObjectApiResponse({'took': 448, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 50, 'relation': 'eq'}, 'max_score': 0.8312477, 'hits': [{'_index': 'wiki-index', '_id': '26576', '_score': 0.8312477}, {'_index': 'wiki-index', '_id': '1045', '_score': 0.75957215}, {'_index': 'wiki-index', '_id': '34133', '_score': 0.7433088}, {'_index': 'wiki-index', '_id': '41826', '_score': 0.7372289}, {'_index': 'wiki-index', '_id': '46560', '_score': 0.7371785}, {'_index': 'wiki-index', '_id': '38066', '_score': 0.7346137}, {'_index': 'wiki-index', '_id': '31922', '_score': 0.7322619}, {'_index': 'wiki-index', '_id': '29886', '_score': 0.724103}, {'_index': 'wiki-index', '_id': '24350', '_score': 0.72298753}, {'_index': 'wiki-index', '_id': '35316', '_score': 0.7224528}, {'_index': 'wiki-index', '_id': '45653', '_score': 0.7212582}, {'_index': 'wiki-index', '_id': '19253', '_score': 0.7211274}, {'_index': 'wiki-index', '_id': '4291',